# DeepLabCut Notebook

Import python libraries needed for this notebook

In [1]:
# Restart Kernel before running this cell if you've run other matplotlib commands
%matplotlib qt

In [1]:
try:
    import deeplabcut
    import tkinter
    from tkinter import filedialog
    
    print(f'Using DeepLabCut version: {deeplabcut. __version__}')

except:
    print("Please run the notebook in in your local environment")

Loading DLC 3.0.0rc9...


/Users/atanugiri/miniconda3/envs/DEEPLABCUT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using DeepLabCut version: 3.0.0rc9


### You might have to import torch to use PyTorch engine ###

In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("cuDNN Version:", torch.backends.cudnn.version())
    print("cuDNN Enabled:", torch.backends.cudnn.enabled)

PyTorch version: 2.7.1
CUDA Available: False


### Check PyTorch in DeepLabCut ###

In [3]:
if torch.cuda.is_available():
    print("DeepLabCut is using PyTorch with GPU support.")
else:
    print("DeepLabCut is using PyTorch on CPU.")

DeepLabCut is using PyTorch on CPU.


Start by selecting the list of videos to be included in the model. You could manually type the full path of each video in a python list as argument of the deeplabcut.create_new_project() function, like so:

:::{note}
Windows users need to use the double backslash for path directories or a python raw filestring.
:::

Instead, we use ```tkinter``` to open a file dialoge and save the file paths in a python list called ```videolist```: 

In [ ]:
video_files = filedialog.askopenfilenames(title='Choose new video files to analyze in DeepLabCut:')
videolist = list(video_files)

print(f'{len(videolist)} videos selected:')
for i in range(len(videolist)): 
    print(videolist[i])

Now we create a new project using the video paths in ```videolist```, give the project a name and set a few parameters:

## Create new DLC Project

In [15]:
from pathlib import Path

video = Path("/Users/atanugiri/Downloads/DeepLabCutProjects/data/BlackAnimals/ToyLight/SplitVideos/ToyLight_10_16_23_P_BlackCement.mp4")
working_dir = Path("/Users/atanugiri/Downloads/DeepLabCutProjects")

# sanity checks
print("Video exists? ", video.exists())
print("Working dir exists? ", working_dir.exists())

config_path = deeplabcut.create_new_project(
    "DLC-BlackFoodAndToyLight",
    "Atanu",
    [str(video)],
    working_directory=str(working_dir),
    copy_videos=True,
    multianimal=False,
)
print("Created:", config_path)


Video exists?  True
Working dir exists?  True
Created "/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/videos"
Created "/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/labeled-data"
Created "/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/training-datasets"
Created "/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/dlc-models"
Copying the videos
/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/videos/ToyLight_10_16_23_P_BlackCement.mp4
Generated "/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/config.yaml"

A new project with name DLC-BlackFoodAndToyLight-Atanu-2025-08-19 is created at /Users/atanugiri/Downloads/DeepLabCutProjects and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.


:::{note}
You can load existing DeepLabCut projects by specifying the config_path as below:
:::

## Add new videos

In [16]:
from pathlib import Path

base1 = Path("/Users/atanugiri/Downloads/DeepLabCutProjects/data/BlackAnimals/FoodLight/SplitVideos").expanduser()
base2 = Path("/Users/atanugiri/Downloads/DeepLabCutProjects/data/BlackAnimals/ToyLight/SplitVideos").expanduser()

names1 = [
    "FoodLight_9_9_24_P_Azul.mp4",
    "FoodLight_9_9_24_P_Navy.mp4",
    "FoodLight_9_9_24_P_Tan.mp4",
    "FoodLight_9_9_24_Y_Cyan.mp4",
    "FoodLight_9_9_24_Y_Orange.mp4"
]

names2 = [
    "ToyLight_10_16_23_P_Flugume.mp4",
    "ToyLight_10_16_23_P_GammaRay.mp4",
    "ToyLight_10_16_23_P_OffWhite.mp4",
    "ToyLight_10_16_23_P_ShatteredBackboard.mp4"
]

new_videos = [str(base1 / n) for n in names1] + [str(base2 / n) for n in names2]

# add to an existing project:
deeplabcut.add_new_videos(config_path, new_videos, copy_videos=True, extract_frames=False)


Copying the videos
Copying the videos
Copying the videos
Copying the videos
Copying the videos
Copying the videos
Copying the videos
Copying the videos
Copying the videos
New videos were added to the project! Use the function 'extract_frames' to select frames for labeling.


### Check the Backend (TensorFlow or PyTorch)

In [13]:
try:
    from tensorflow.python.client import device_lib
    print("Using TensorFlow backend.")
except ImportError:
    try:
        import torch
        print("Using PyTorch backend.")
    except ImportError:
        print("No supported backend found. Ensure TensorFlow or PyTorch is installed.")

Using TensorFlow backend.


## Configure Project

Now that a new project has been created with a specific directory structure and configuration file, we need to tweak some parameters to tailor the bodyparts we want to track:

In [3]:
config_path = '/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/config.yaml'
print(config_path)


/Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/config.yaml


In [ ]:
import webbrowser
webbrowser.open(config_path)
print('Please edit bodyparts list to be tracked')

In [ ]:
with open(config_path, 'r') as file:
    config_content = file.read()

Once happy with all ```bodyparts```, ```skeleton:``` and ```numframes2pick:``` settings, start extracting frames to label:


## Select Frames to Label

In [ ]:
deeplabcut.extract_frames(
    config_path, mode='automatic', 
    algo='uniform', 
    crop='GUI', # <--- THIS IS KEY: Tells DLC you want to crop
    userfeedback=True # <--- THIS IS KEY: Ensures an interactive GUI for selection
)

# deeplabcut.extract_frames(config_path, "manual")

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: /Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/videos/ToyLight_10_16_23_P_BlackCement.mp4 ?


yes/no yes
The directory already contains some frames. Do you want to add to it?(yes/no):  yes


Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 1366.07  seconds.
Do you want to extract (perhaps additional) frames for video: /Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/videos/FoodLight_9_9_24_P_Azul.mp4 ?


## Label Frames (Do it in GUI)

In [17]:
deeplabcut.label_frames(config_path)


OSError: No supported images were found in /Users/atanugiri/Downloads/DeepLabCutProjects/DLC-BlackFoodAndToyLight-Atanu-2025-08-19/labeled-data/FoodLight_9_9_24_P_Azul.

You can plot your labeled frames to check your annotation accuracy.

In [4]:
deeplabcut.check_labels(config_path)

Creating images with labels by Atanu.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 83.62it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## Create Training Dataset

In [8]:
deeplabcut.create_training_dataset(config_path)


[(0.95,
  3,
  (array([ 45,  59,   7,  50,  92,  27, 131, 137, 122,   8, 111,  16,  63,
           76, 123,  97, 104, 110,  33,  91,  90,  22, 102,  24,   2,  51,
           26,  71,  18,  10,  56,  43, 109,  48, 107,  83,  60, 106,  89,
           78,  44,  30,  62, 121, 114,  73,  95,  84, 134,  13,  54,  94,
          120, 118,  15,  68,  40,  61,  86,  66,   3,  52, 112, 116,   6,
          124,  12,  85, 136, 127, 126,  11,  93,  98,  41, 100,   1,  96,
          129,  42,   4, 113,  17,  38,   5,  53, 133, 108,   0,  34,  28,
           55,  75,  35,  23,  74,  31, 101,  57, 119,  65,  32, 128,  14,
          105,  19,  29,  49, 125,  99,  82,  64, 139,  79,  69, 138,  80,
          115,  20, 135,  72,  77,  25,  37,  81, 130,  46, 132,  39,  58,
           88,  70,  87]),
   array([ 36,  21,   9, 103,  67, 117,  47])))]

## Train The Network

In [9]:
deeplabcut.train_network(
    config_path,
    shuffle=3,
    trainingsetindex=0,
    device="cuda:0",
    max_snapshots_to_keep=5,
    displayiters=100,
    save_epochs=5,
    epochs=200,
)


Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: C:\DeepLabCutProjects\DLC-WhiteAnimals-Atanu-2025-07-23
  pose_config_path: C:\DeepLabCutProjects\DLC-WhiteAnimals-Atanu-2025-07-23\dlc-models-pytorch\iteration-0\DLC-WhiteAnimalsJul23-trainset95shuffle3\train\pytorch_config.yaml
  bodyparts: ['Corner1', 'Corner2', 'Corner3', 'Corner4', 'Head', 'Neck', 'Midback', 'Lowerback', 'Tailbase']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    freeze_bn_stats: False
    freeze_bn_weights: False
  backbone_outp

## Evaluate the Trained Network

In [9]:
deeplabcut.evaluate_network(config_path, Shuffles=[3], plotting=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 50.43it/s]


Evaluation results for DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle2_snapshot_200-results.csv (pcutoff: 0.3):
train rmse            45.36
train rmse_pcutoff     3.38
train mAP             59.74
train mAR             68.42
test rmse             44.46
test rmse_pcutoff      8.18
test mAP              63.89
test mAR              72.86
Name: (0.95, 2, 200, -1, 0.3), dtype: float64


## Analyze new Videos

#### Test videos

In [14]:
videolist = [r"C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4",
            r"C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4",
            r"C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4"]
# destfolder = r"C:\DeepLabCutProjects\data\FoodLight\DlcDataPytorch"

deeplabcut.analyze_videos(config_path, videos=videolist, shuffle=3, gputouse="cuda:0", save_as_csv=True)


Analyzing videos with C:\DeepLabCutProjects\DLC-WhiteAnimals-Atanu-2025-07-23\dlc-models-pytorch\iteration-0\DLC-WhiteAnimalsJul23-trainset95shuffle3\train\snapshot-best-100.pt
Starting to analyze C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4
Video metadata: 
  Overall # of frames:    2417
  Duration of video [s]:  96.72
  fps:                    24.99
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|██████████████████████████████████████████████████████████████████████████████| 2417/2417 [00:55<00:00, 43.76it/s]


Saving results in C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4
Video metadata: 
  Overall # of frames:    2390
  Duration of video [s]:  95.64
  fps:                    24.99
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|█████████████████████████████████████████████████████████████████████████████▉| 2389/2390 [00:56<00:00, 42.42it/s]


Saving results in C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4
Video metadata: 
  Overall # of frames:    2334
  Duration of video [s]:  93.40
  fps:                    24.99
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|██████████████████████████████████████████████████████████████████████████████| 2334/2334 [00:56<00:00, 41.21it/s]


Saving results in C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.



'DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100'

#### All videos in a directory

In [21]:
import os

# base data path
base_path = r"C:\DeepLabCutProjects\data"

# List of subfolders to process
conditions = ['ToyOnlyWhite', 'ToyLightWhite']

# Loop over all conditions
for condition in conditions:
    print(f"\nProcessing: {condition}")

    video_dir = os.path.join(base_path, condition, "SplitVideos")
    destfolder = os.path.join(base_path, condition, "DlcDataPytorch")
    os.makedirs(destfolder, exist_ok=True)

    # Get list of video files
    videolist = [
        os.path.join(video_dir, f)
        for f in os.listdir(video_dir)
        if f.endswith(('.mp4', '.avi'))
    ]

    # Skip if no videos
    if not videolist:
        print(f"No videos found in {video_dir}, skipping.")
        continue

    # Analyze
    print(f"Analyzing {len(videolist)} videos, saving to {destfolder}")
    deeplabcut.analyze_videos(
        config_path,
        videos=videolist,
        shuffle=3,
        gputouse="cuda:0",
        save_as_csv=True,
        destfolder=destfolder
    )

print("\nAll batches submitted for analysis.")



Processing: ToyOnlyWhite
Analyzing 47 videos, saving to C:\DeepLabCutProjects\data\ToyOnlyWhite\DlcDataPytorch
Analyzing videos with C:\DeepLabCutProjects\DLC-WhiteAnimals-Atanu-2025-07-23\dlc-models-pytorch\iteration-0\DLC-WhiteAnimalsJul23-trainset95shuffle3\train\snapshot-best-100.pt
Video C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Doc.mp4 already analyzed at C:\DeepLabCutProjects\data\ToyOnlyWhite\DlcDataPytorch\ToyOnlyExcitatory_2_26_25_S1Y_DocDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle!
Video C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Dopey.mp4 already analyzed at C:\DeepLabCutProjects\data\ToyOnlyWhite\DlcDataPytorch\ToyOnlyExcitatory_2_26_25_S1Y_DopeyDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle!
Video C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Grumpy.mp4 already analyzed at C:\DeepLabCutProjects\data\ToyOnlyWhite\Dl

100%|████████████████████████████████████████████████████████████████████████████| 30017/30017 [12:00<00:00, 41.66it/s]


Saving results in C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLightExcitatory_2_24_25_S2Y_HappyDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLightExcitatory_2_24_25_S2Y_HappyDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\ToyLightWhite\SplitVideos\ToyLightExcitatory_2_24_25_S2Y_Sleepy.mp4
Video metadata: 
  Overall # of frames:    30017
  Duration of video [s]:  1200.68
  fps:                    25.0
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|████████████████████████████████████████████████████████████████████████████| 30017/30017 [12:10<00:00, 41.11it/s]


Saving results in C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLightExcitatory_2_24_25_S2Y_SleepyDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLightExcitatory_2_24_25_S2Y_SleepyDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\ToyLightWhite\SplitVideos\ToyLightInhibitory_1_22_25_S1P_A.mp4
Video metadata: 
  Overall # of frames:    30015
  Duration of video [s]:  1200.60
  fps:                    25.0
  resolution:             w=360, h=288

Running pose prediction with batch size 8


...

Saving results in C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S2Y_5DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S2Y_5DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\ToyLightWhite\SplitVideos\ToyLight_2_7_25_S2Y_6.mp4
Video metadata: 
  Overall # of frames:    30020
  Duration of video [s]:  1200.80
  fps:                    25.0
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|████████████████████████████████████████████████████████████████████████████| 30020/30020 [11:56<00:00, 41.87it/s]


Saving results in C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S2Y_6DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S2Y_6DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
Starting to analyze C:\DeepLabCutProjects\data\ToyLightWhite\SplitVideos\ToyLight_2_7_25_S3Y_9.mp4
Video metadata: 
  Overall # of frames:    30020
  Duration of video [s]:  1200.80
  fps:                    25.0
  resolution:             w=360, h=288

Running pose prediction with batch size 8


100%|████████████████████████████████████████████████████████████████████████████| 30020/30020 [11:56<00:00, 41.89it/s]


Saving results in C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S3Y_9DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100.h5 and C:\DeepLabCutProjects\data\ToyLightWhite\DlcDataPytorch\ToyLight_2_7_25_S3Y_9DLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


All batches submitted for analysis.


## Filter Pose Data

#### Test videos

In [17]:
deeplabcut.filterpredictions(config_path, videolist, shuffle=3, filtertype='median', p_bound=0.05)

Filtering with median model C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4
Saving filtered csv poses!


#### All videos in a directory

In [11]:
import os

# base data path
base_path = r"C:\DeepLabCutProjects\data"

# List of subfolders to process
conditions = ['ToyOnlyWhite']

# Loop over all conditions
for condition in conditions:
    print(f"\nProcessing: {condition}")

    video_dir = os.path.join(base_path, condition, "SplitVideos")

    # Get list of video files
    videolist = [
        os.path.join(video_dir, f)
        for f in os.listdir(video_dir)
        if f.endswith(('.mp4', '.avi'))
    ]

    # Skip if no videos
    if not videolist:
        print(f"No videos found in {video_dir}, skipping.")
        continue

    # Analyze
    deeplabcut.filterpredictions(
        config_path,
        videolist,
        shuffle=3,
        save_as_csv=True
    )

print("\nAll batches submitted for analysis.")


Processing: ToyOnlyWhite
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Doc.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Dopey.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Grumpy.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Sneezy.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S2Y_Bashful.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S2Y_Happy.mp4
Saving filtered csv poses!
Filtering with median model C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExc

In [ ]:
deeplabcut.analyzeskeleton(config_path, videolist, videotype='.mp4', shuffle=1, trainingsetindex=0, save_as_csv=False, destfolder=None)

## Create labeled videos

In [20]:
import os

# video_dir = r"C:\DeepLabCutProjects\data\FoodOnly\SplitVideos"
# videolist = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]

# videolist = [r"C:\DeepLabCutProjects\DLC-Atanu-2025-06-10\videos\FoodOnly\FoodOnly_8_28_24_S3P_Cyan_Trial1.mp4"]

# deeplabcut.create_labeled_video(config_path, videolist, draw_skeleton=False, filtered=True)

deeplabcut.create_labeled_video(
    config_path,
    videolist,
    shuffle=3,
    filtered=True,
    fastmode=False,
    save_frames=False,
    displayedbodyparts=['Head', 'Neck', 'Midback', 'Lowerback', 'Tailbase']
)


INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 360x288 -pix_fmt rgba -framerate 24.9852410891309 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y 'C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4'


Starting to process video: C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4
Loading C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4 and data.
Duration of video [s]: 96.74, recorded with 24.99 fps!
Overall # of frames: 2417 with cropped frame dimensions: 360 288
Generating frames and creating video.


100%|██████████████████████████████████████████████████████████████████████████████| 2417/2417 [00:58<00:00, 41.23it/s]
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 360x288 -pix_fmt rgba -framerate 24.990258602450503 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y 'C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4'


Labeled video C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4 successfully created.
Starting to process video: C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4
Loading C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4 and data.
Duration of video [s]: 95.64, recorded with 24.99 fps!
Overall # of frames: 2390 with cropped frame dimensions: 360 288
Generating frames and creating video.


100%|██████████████████████████████████████████████████████████████████████████████| 2389/2389 [00:56<00:00, 42.08it/s]
INFO:matplotlib.animation:MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 360x288 -pix_fmt rgba -framerate 24.98774667654739 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y 'C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4'


Labeled video C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4 successfully created.
Starting to process video: C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4
Loading C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4 and data.
Duration of video [s]: 93.41, recorded with 24.99 fps!
Overall # of frames: 2334 with cropped frame dimensions: 360 288
Generating frames and creating video.


100%|██████████████████████████████████████████████████████████████████████████████| 2334/2334 [00:55<00:00, 42.32it/s]

Labeled video C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmedDLC_Resnet50_DLC-WhiteAnimalsJul23shuffle3_snapshot_100_filtered_p30_labeled.mp4 successfully created.


[True, True, True]

### Plot Trajectories

In [ ]:
deeplabcut.plot_trajectories(config_path, videolist)

# Optional Active Learning -> Network Refinement

Load new videos to analyze and/or merge to the project:

## Extract outlier frames

Now this is the interesting part. Instead of including more videos to the project directly, and extracting frames as usual with kmeans, we are taking advantage of the previous model to tell us what frames exactly to label. This active learning step helps us recognize the shortcomings of our model and correct it in a targeted manner.

In [ ]:
deeplabcut.extract_outlier_frames(config_path, videolist, outlieralgorithm='uncertain')

## Refine Labels: Augmentation of the Training Dataset

Now that we have extracted new frames, we need to go back and start labeling. Instead of starting from the beginning, though, we are provided the model predictions and have to drag and drop them in place. **Note:** Make sure to remove labels that are not visible, the model will often guess the expected position based on learned geometric constraints. 

In [ ]:
deeplabcut.refine_labels(config_path)

You can again plot your labeled frames to check annotation accuracy.

In [ ]:
deeplabcut.check_labels(config_path)

At this point you could get an error message like [this](https://github.com/DeepLabCut/DeepLabCut/issues/232) telling you that saving the video path failed. In this case, you need to add the new video paths manually for DLC to include these in the new training set. You can either add them by hand, writing in the config.yaml file in the same format as the first video paths (see [here](https://github.com/DeepLabCut/DeepLabCut/issues/663#issuecomment-619274975)), or you can run the following command to add the list of videos to your config file:

If the permission error persists, try starting a new anaconda terminal as administrator (right click > run as administrator) and then starting jupyter notebook with elevated privileges.

## Merge Datasets

After refining all outlier frames extracted above, merge the datasets to combine old and new labels in your project. 

In [ ]:
deeplabcut.merge_datasets(config_path)

**Note:** Make sure that the new videos have been included in the config.yaml file without permission issues (see above).

### Re-Train Network

When training a new model with an expanded dataset, you could either choose to start fresh with new data, or use the previous model as pre-trained network for your next model. Although not yet extensively verified, lets belief that transfer learning at least won't harm the new model. 

### Example of own pre-trained model
init_weights: D:\FacialExpression\old-DLC-Project\dlc-models\iteration-0\DLCApr14-trainset95shuffle1\train\snapshot-1030000

## Start over again...